In [ ]:
import keras
import tensorflow as tf
import sys
import os
from keras.layers import *
from keras import *
from keras.models import Model
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.regularizers import l2 
from keras.optimizers import SGD, Adam, RMSprop
from keras.layers import Conv1D, Conv2D, Dense, MaxPooling1D, Flatten, Dropout, BatchNormalization, LSTM, Bidirectional
from keras.callbacks import ModelCheckpoint, EarlyStopping, LearningRateScheduler, CSVLogger
from keras.models import model_from_json
from keras.models import model_from_yaml
from keras.initializers import RandomUniform, glorot_uniform, TruncatedNormal
from keras.models import Sequential
import keras.backend as K
from random import shuffle

import pandas as pd 
from group_norm import GroupNormalization

import numpy as np
import h5py
import scipy.io
seed = 123
np.random.seed(seed) # for reproducibility


#scikit learn
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import jaccard_similarity_score
from sklearn.metrics import f1_score
from matplotlib import pyplot as plt
import matplotlib.pyplot as plt
%matplotlib inline

from keras import losses
from sklearn.metrics import confusion_matrix,recall_score,matthews_corrcoef,roc_curve,roc_auc_score,auc
from sklearn.metrics import precision_recall_fscore_support
from scipy import interp

In [ ]:
from sklearn.metrics import average_precision_score
from sklearn.metrics import roc_curve, auc
import math
from operator import truediv
from sklearn.metrics import confusion_matrix
from keras.metrics import binary_accuracy
import itertools
from keras.callbacks import CSVLogger
from keras.callbacks import EarlyStopping 
from keras.callbacks import ModelCheckpoint
from keras.callbacks import ReduceLROnPlateau
import pickle
from keras.constraints import unit_norm
from keras.constraints import max_norm
from keras.utils.vis_utils import plot_model

In [ ]:
fold= 10

for j in range(fold):    
    
    # Train and Validation datasets in folds
    train_features = np.load('folds/'+str(j)+'_x_train.npy')
    train_labels = np.load('folds/'+str(j)+'_y_train.npy')

    validation_x = np.load('folds/'+str(j)+'_valid_X.npy')
    validation_y = np.load('folds/'+S+'/'+str(j)+'_valid_y.npy')
    

    print('\nFold ',j)
    
    # model save function 
    class CustomModelCheckpoint(keras.callbacks.Callback):
        def __init__(self, model, path):
            self.path = path
            self.model_for_saving = model
            
    
    # Defining Model for training
    model = Sequential()
    model.add(Conv1D(16, 5, activation='elu', input_shape=(51, 4), kernel_regularizer = regularizers.l2(1e-3), bias_regularizer = regularizers.l2(1e-3)))
    model.add(GroupNormalization(groups = 4,axis=-1))
    model.add(MaxPooling1D(4,strides=2))

    model.add(Conv1D(16, 4, strides=1, activation='elu', kernel_regularizer = regularizers.l2(1e-3), bias_regularizer = regularizers.l2(1e-3)))
    model.add(GroupNormalization(groups = 4,axis=-1))
    model.add(MaxPooling1D(4,strides=2))
    model.add(Dropout(0.35))

    model.add(Flatten())

    model.add(Dense(32, activation='elu', kernel_regularizer = regularizers.l2(1e-4),bias_regularizer = regularizers.l2(1e-4)))
    model.add(Dense(1, activation='sigmoid'))
    

    model.compile(loss='binary_crossentropy', optimizer= SGD(momentum = 0.95, lr = 0.003), metrics=[binary_accuracy]);

    model.summary()
    
    # save model and architecture to single file
    model_name='Yeast_Model6540_'+str(j)
    print(model_name)

    result_folder= 'Results/'
    if not os.path.exists(result_folder+model_name):
        os.makedirs(result_folder+model_name)
    model_results_folder=result_folder+model_name+ '/'
    best_weights = model_results_folder + 'best_weights.h5'

    model_arch = model_results_folder + 'model.json'

    checkpointsString = model_results_folder + 'weights.{epoch:02d}-{val_loss:.2f}.hdf5'
    
    callbacks = [CustomModelCheckpoint(model, checkpointsString)]
    early_stop = EarlyStopping(monitor='val_binary_accuracy', patience=30,restore_best_weights=True) 
    best_model = ModelCheckpoint(best_weights, monitor='val_binary_accuracy', verbose=0, save_best_only=True, save_weights_only=True, mode='max',period=1)
    reduct_L_rate = ReduceLROnPlateau(monitor='val_loss',factor=0.1, patience=20)
    cbacks = [early_stop,best_model,reduct_L_rate,CustomModelCheckpoint(model, checkpointsString)]
    
    model_json = model.to_json()
    with open(model_arch, "w") as json_file:
        json_file.write(model_json)
    
    history = model.fit(train_features,train_labels,epochs=100, batch_size=32, validation_data=(validation_x, validation_y), callbacks=cbacks)
    
    model.save_weights(best_weights)
    